# The OpenMC C-API

Welcome to the multiphysics part of the OpenMC workshop. This section will show the basics of using OpenMC's C-API for performing multiphysics coupling. OpenMC's C-API allows you to interact with an OpenMC simulation in-memory by changing cell temperatures and densities, adding/removing tallies, and much more - all without re-initializing OpenMC between multiphysics feedback steps.

# Problem Description

This example will couple OpenMC to a basic thermal-fluids solver for a UO2 pincell in water. OpenMC will send a volumetric heat source to the thermal-fluids solver, which will use this heat source to compute temperatures and densities that will be sent back to OpenMC. The two physics (OpenMC, thermal-fluids) will then be iterated several times in-memory.
We will divide the problem into $N$ axial cells in both OpenMC and the T/H app, with coupling between layer $i$ in OpenMC and layer $i$ in the T/H app. We will not couple solid _density_ to this problem because to properly conserve the fissile mass, this would require re-generation of the geometry (i.e. larger or smaller pin radii) with each coupled physics iteration. 

<img src="setup.png" alt="drawing" width="500"/>

To keep our main focus on OpenMC, all of the T/H physics is located in a separate Python module, `th_backend`. This module contains functions that compute the fluid temperature, fluid density, and solid temperature based on steady state energy conservation. 

<div class="alert alert-block alert-info">
Do NOT use the th_backend.py for any real simulations -- the physical models are extremely simplified.
</div>

We will couple OpenMC and our thermal-fluids application with Picard iteration. Our simulation will be a steady-state simulation, so we will iterate between OpenMC and our thermal-fluids application until the solution (power, temperatures, densities) has stopped changing with iteration.

<img src="picard.png" alt="drawing" width="500"/>

### Changing Temperatures and Densities in OpenMC

We require separate cells so that OpenMC can apply a histogram-type resolution for temperature and density feedback. Changes to temperature and density are easiest to apply by changing these quantities on the cell objects; as long as a cell has a unique ID/instance pair, it's temperature/density/tally can be distinguished from other cells with different ID/instance pairs.

## Model

In [ ]:
import openmc
import th_backend
import numpy as np
import matplotlib.pyplot as plt

model = openmc.Model()

We will use the existing pincell model in OpenMC. The thermal-hydraulic solver also needs to know some quantities about the dimensions, so we fetch those parameters and assign them to local variables for convenience.

In [ ]:

model.plot(width=(2, 2))

In [ ]:
# find the cylinder radii and pitch
cylinder_radii = []
xplane_x0 = []
surfaces = model.geometry.get_all_surfaces()
for s in surfaces:


cylinder_radii.sort()
xplane_x0.sort()

In [ ]:
R = cylinder_radii[1]                 # outer radius of the pincell (cm)
Rf = cylinder_radii[0]                # outer radius of the pellet (cm)
pitch = xplane_x0[1] - xplane_x0[0]   # pitch between pincells (cm)
height = 381.0                        # height of the pincell (cm)

power = 80.0e3                       # total power produced by pincell (W)
T_inlet = 573.0                       # inlet water temperature (K)

Next, specify the number of layers we want to build in our model. Note that the more layers you add, the better the resolution will be for the temperature, density, and tally feedback - but the more particles which will be needed as you make each tally region smaller and smaller.

In [ ]:
N = 20                    # number of coupling layers         
H = height / N            # height of each coupling layer

# Geometry

Next, let's define the geometry. As our geometry currently stands, it is infinite in the $z$-direction and also has no "cell resoluion" in the vertical direction. In other words, because temperatures and densities will be applied to our cells, we want to make sure that there are more unique cells in the vertical direction so that the cells can be set to a spatially-varying temperature and density.

The most efficient way to do this is to simply put our universe, `model.geometry.root_universe`, that we pulled from the pre-built examples into a lattice. 

In [ ]:


prism = openmc.model.RectangularPrism(width = pitch, height = pitch, boundary_type = 'reflective')
bottom = openmc.ZPlane(z0=0, boundary_type='vacuum')
top = openmc.ZPlane(z0=height, boundary_type='vacuum')



In [ ]:
root.plot(width=(pitch*2, pitch*2))

We have repeated the same cells (fuel, clad, water box) in each layer of the lattice. OpenMC differentiates between these cells, once they are placed into the lattice, through the notion of cell *instances*. Because we loaded this model from a pre-built example, we need to have OpenMC re-load/compute the instancing information.

### Tallies

In order to provide a power to our T/H app, we need to extract the fission power from OpenMC. We therefore need to add tallies for the recoverable fission energy that is deposited in each layer. In order to normalize these heat tallies into units of W/cm$^3$, we will also need to tally the total fission energy deposited. In other words, a global heating tally allows us to compute $\textcolor{red}{S}$.

Heating tally: $r_g \left\lbrack\frac{\text{eV}}{\text{src}}\right\rbrack * \textcolor{red}{S} \left\lbrack\frac{\text{src}}{\text{s}}\right\rbrack = p \left\lbrack\frac{J}{\text{s}}\right\rbrack * \frac{1}{1.602\times10^{-19}} \left\lbrack\frac{\text{eV}}{\text{J}}\right\rbrack $

We then normalize our local heating tally as

Local heating tally: $r_l \left\lbrack\frac{\text{eV}}{\text{src}}\right\rbrack * \textcolor{red}{S} \left\lbrack\frac{\text{src}}{\text{s}}\right\rbrack * \frac{1}{V} \left\lbrack\frac{1}{\text{cm$^3$}}\right\rbrack * 1.602\times10^{-19} \left\lbrack\frac{\text{J}}{\text{eV}}\right\rbrack \rightarrow \dot{q} \left\lbrack\frac{W}{\text{cm$^3$}}\right\rbrack $

$\frac{r_l \left\lbrack\frac{\text{eV}}{\text{src}}\right\rbrack}{r_g \left\lbrack\frac{\text{eV}}{\text{src}}\right\rbrack} * p \left\lbrack\frac{J}{\text{s}}\right\rbrack * \cancel{\frac{1}{1.602\times10^{-19}} \left\lbrack\frac{\text{eV}}{\text{J}}\right\rbrack} * \frac{1}{V} \left\lbrack\frac{1}{\text{cm$^3$}}\right\rbrack * \cancel{1.602\times10^{-19} \left\lbrack\frac{\text{J}}{\text{eV}}\right\rbrack} \rightarrow \dot{q} \left\lbrack\frac{W}{\text{cm$^3$}}\right\rbrack $

In [ ]:

model.tallies = [fission_tally]

Now come the settings. First, we will specify the initial source. Here we will just use a uniform source over all of the fissionable regions in the problem - i.e. the cylinder fuel pellet. We also specify some batch and particle settings.

In [ ]:


model.settings.particles = 1000
model.settings.inactive = 200
model.settings.batches = 500

### How OpenMC Uses Temperature

There are three ways to define temperature in OpenMC, and a fourth option is coming soon:

1. Set a default temperature, using `settings.temperature['default']`
2. Set the temperature of an `openmc.Material`
3. Set the temperature of an `openmc.Cell`
4. Read temperature from a superimposed mesh (coming soon)

In order of precedence, OpenMC attempts to find temperatures from the bottom of the list, going to the top. For example, in our multiphysics calculation we can use `settings.temperature['default']` to easily set a uniform initial temperature for the entire domain. But when we want to set a spatially-dependent temperature, we'll set those temperatures using the cell objects. We could have used the material approach, but this would require us to create copies of the materials for each location they appear in the model, which for large problems can be memory intensive.

### Temperatures Not in the LIbrary

We need to set the method by which OpenMC determines which cross sections to read if a corresponding library is not available on your machine. For example, you may want to change the fuel to 725 K, but only have libraries at 600 K and 900 K. There are two available methods:

- `interpolation`, which means that if a cell is at temperature $T$, that the microscopic cross section is evaluated based on a stochastic interpolation between the cross sections at $T_i$ and $T_{i+1}$, where $T_i$ and $T_{i+1}$ are the temperatures that bound $T$ for which cross section data is loaded.
- `nearest`, which will select the closest available library.

We want to be sure to load cross section data over a fairly wide range in temperatures - we're not sure exactly how hot the fuel is going to be for the coupled case until we run it! So we can set a `range` from 294 K to something high - say, 3000 K. This should hopefully cover all the temperatures we encounter in our coupled run. This is necessary because OpenMC will load all the cross sections into memory at the start of the run (for performance reasons, we don't check each time a collision occurs that the library contains the relevant data needed at that temperature - we simply move straight to evaluating).

### How OpenMC Uses Density

There are two ways to define density in OpenMC:

1. Set the density of an `openmc.Material`
2. Set a multiplier on the density of an `openmc.Cell` filled by a material

In this notebook, we will use the second option because this allows us to have just one definition for the fuel material, but scale its density to the value desired in each cell.

In [ ]:
model.settings.temperature = {}

# Running OpenMC: All-at-Once

In the earlier parts of this workshop, we ran OpenMC with the command `model.run()`, like this:

In [ ]:
statepoint = model.run()

# Running OpenMC : Step-by-Step

OpenMC's C-API contains python bindings to C++ functions that allow you to individually control modular pieces of an overall OpenMC calculation, so that you no longer need to run an entire OpenMC calculation all at once.
Running OpenMC with the C-API includes the following major steps:

1. Initializing OpenMC (loading cross section data, building the geometry, etc.)
2. Clearing any tally data
3. Running OpenMC (stochastic transport for the batches, accumulating tallies, etc.)
4. Finalizing OpenMC (freeing dynamically-allocated memory, closing the parallel communicator, etc.)

This function reads all of the XML files associated with this run and loads cross section data. Next, clear any tally data:

Next, let's run OpenMC for the batches we have specified.

To wrap up the simulation, finalize OpenMC:

# Running OpenMC: Multiphysics iteration

As you can see from the example above, the C-API lets us separate the overall OpenMC calculation into separate steps, where we can insert other behavior between steps, while also skipping the same initialization and finalization steps we don't need for in-memory coupling. In this section, we build a simple multiphysics example with temperature, density, and heat source iteration with the thermal-fluids solver. First, let's specify the number of iterations we'd like to perform.

In [ ]:
n_iterations = 5

As we iterate between Monte Carlo transport and thermal-fluids, we can improve the stability by _relaxing_ the heat source computed by OpenMC - all that means is that in iteration $j$, we compute the heat source as an average of the heat source _just_ computed by OpenMC and that of the previous iterate:

$q_j=(1-\alpha)q_{j-1}+\alpha \Phi_j$

where $\Phi$ indicates the Monte Carlo "operation." All we need to do here is define $\alpha$, which we'll just take as 0.5:

In [ ]:
alpha = 0.5

A classic difficulty with multiphysics calculations is mapping data between the Monte Carlo code and the thermal-fluids code - what cell in OpenMC corresponds to the node/element/quadrature point in my thermal-fluids application? OpenMC's C-API provides several convenient functions to find the cell and material at a given position in space. Although in this example we built the thermal-fluids mesh to exactly match OpenMC's cell division, we'll highlight how the C-API could be used to determine the mapping in an automated fashion. 

To do this, we'll define coordinates in each of the coupling regions - the fluid, the clad, and the fuel. Under the hood, OpenMC uses its cell lookup routines to find which cell the point is in - because our thermal-fluids data computes volume-averaged quantities, we just need to identify _one_ point in each layer.

In [ ]:
# z-coordinates of each layer, H is the layer height
cell_centers = np.linspace(H / 2.0, height - H / 2.0, N)

# an x-coordinate in each of the various regions
x_fuel = 0.0
x_clad = Rf + 0.001 # just a little bit outside the pellet surface
x_fluid = R + 0.001 # just a little bit outside the pincell surface

## The Multiphysics Loop

For our multiphysics calculation, we want to exchange data with our thermal-fluids application in-memory. The alternative approach - sometimes referred to as "external coupling," would instead faciltate multiphysics by 

1. Initializing, running, and finalizing OpenMC
2. Writing an OpenMC output file
3. Using a script to parse OpenMC's output file and write a new thermal-fluids app input file
4. Initializing, running, and finalizing the thermal-fluids app
5. Using a script to parse the thermal-fluids app's output file and write a new OpenMC input file

A downside of this approach is that many steps are repeated for each iteration even though they're not required - such as reading cross section data. A second downside of the external coupling approach is that you need to write scripts to handle I/O.

### A Convenient Context Manager

Previously, we ran the four steps of an OpenMC simulation separately  - `init()`, `reset()`, `run()`, and `finalize()`. The C-API offers a convenient context manager that actually handles the initialization and finalization for you, such that all you need to do to run OpenMC multiple times in memory is:

      with openmc.lib.run_in_memory():
          for i in range(n_iterations):
              openmc.lib.reset()
              openmc.lib.run()
    
              # DO STUFF - such as multiphysics!

Below is our code for multiphysics feedback in OpenMC. Let's walk through it step-by-step.

1. First, we reset the tallies (because now we are running OpenMC in a loop, and for this problem we want the tallies to be separate for each run) and then run OpenMC (based on whatever temperatures and densities are set in OpenMC's model).

2. Second, we normalize the kappa fission tally computed by OpenMC, and store the normalized power (with units of Watts) in `q`.

3. Third, we solve our thermal-fluids app by calling the `fluid_temperature`, `fluid_density`, and `solid_temperature` functions we defined at the very beginning of this notebook.

4. Fourth, use the results of the thermal-fluid app to update the temperatures and densities used in OpenMC. For this, we use the C-API functions to first (a) find the cell/material we need to modify based on a coordinate in the cell-of-interest and (b) write the cell/material temperature or density.

### Setting Temperatures and Densities

To find the cell that corresponds to the point `(x_fuel, 0, z)`, we use the function `openmc.lib.find_cell`, which returns a reference to the cell and the instance of that cell. Once we've found the cell, then we can call `set_temperature` for that cell to apply the temperature from our thermal-fluids app. Likewise, once we have the cell ID and instance, we set its density with the `set_density` method.

In [ ]:
model.settings.particles = 80000

q_iterations = []
fluid_temp_iterations = []
solid_temp_iterations = []
fluid_density_iterations = []

with openmc.lib.run_in_memory():
    for i in range(n_iterations):
        openmc.lib.reset()

        openmc.lib.run(output=False)

        # ---- Multiphysics feedback part ---- #
    
        # get the total kappa fission computed by OpenMC over the entire domain
        total_kappa_fission = openmc.lib.tallies[1].mean.sum()

        # power (W) in each layer of the solid
        q = np.zeros(N)

        for j in range(N):
            q[j] = openmc.lib.tallies[1].mean[j].squeeze() / total_kappa_fission

            # to get in units of W, multiply by the total power
            q[j] *= power

            # for greater than the first iteration, relax
            if (i > 0):
                q[j] = (1.0 - alpha) * q_iterations[i - 1][j] + alpha * q[j]

        # compute the fluid temps, fluid densities, and solid temps
        fluid_temps = th_backend.fluid_temperature(q, T_inlet, N)
        fluid_densities = th_backend.fluid_density(fluid_temps, N)
        solid_temps = th_backend.solid_temperature(q, fluid_temps, N, R, Rf, H)
    
        for j in range(N):
            #print("Layer {:3n}:  Percent power: {:5.1f}  Fluid T: {:7.1f}  Solid T: {:7.1f}  Fluid density: {:5.2f}".format(j, q[j] / power * 100, fluid_temps[j], solid_temps[j], fluid_densities[j]))

            z = cell_centers[j]

            # solid temperature
            cell_s, instance_s = openmc.lib.find_cell((x_fuel, 0, z))
            cell_s.set_temperature(solid_temps[j], instance_s)

            # clad temperature (just set to fluid temperature)
            cell_c, instance_c = openmc.lib.find_cell((x_clad, 0, z))
            cell_c.set_temperature(fluid_temps[j], instance_c)

            # get a point in the fluid phase to find the fluid cell
            cell_f, instance_f = openmc.lib.find_cell((x_fluid, 0, z))
            cell_f.set_temperature(fluid_temps[j], instance_f)

            # set the fluid density
            cell_f.set_density(fluid_densities[j], instance_f)
            
        print("Finished Iteration {:3n}".format(i))
        
        # save all of the fields computed from this iteration for plotting later
        q_iterations.append(q)
        fluid_temp_iterations.append(fluid_temps)
        fluid_density_iterations.append(fluid_densities)
        solid_temp_iterations.append(solid_temps)

The above runs OpenMC $N$ times, with the initialization step (where the big OpenMC logo prints and cross-sections are loaded) only performed once at the beginning. Because we use the `openmc.lib.run_in_memory()` context manager, we don't need to worry about initializing and finalizing OpenMC, and instead can focus just on what we want to simulate for each iteration.

Finally, let's plot our multiphysics simulation for all of the iterations.

In [ ]:
normalized_cell_centers = np.array([c / height for c in cell_centers])

fig, axs = plt.subplots(2, 2, figsize=(8,8))

for i in range(n_iterations):
    axs[0, 0].plot(normalized_cell_centers, q_iterations[i], label = 'iteration {:2n}'.format(i))
    axs[0, 0].set_ylabel('$q$ (W)')

    axs[0, 1].plot(normalized_cell_centers, solid_temp_iterations[i], label = 'iteration {:2n}'.format(i))
    axs[0, 1].set_ylabel('$T_s$ (K)')

    axs[1, 0].plot(normalized_cell_centers, fluid_temp_iterations[i], label = 'iteration {:2n}'.format(i))
    axs[1, 0].set_ylabel('$T_f$ (K)')

    axs[1, 1].plot(normalized_cell_centers, fluid_density_iterations[i], label = 'iteration {:2n}'.format(i))
    axs[1, 1].set_ylabel('$\\rho_f$ (g/cm$^3$)')

for ax in axs.flat:
    ax.set(xlabel = 'Normalized Axial Coordinate')
    ticks = np.arange(H / height / 2.0, 1.0, H / height)
    ax.set_xticks([0.0, 0.25, 0.5, 0.75, 1.0])
    ax.set_xticks(ticks, minor = True)
    ax.legend()
    ax.grid()